In [1]:
from manim import *
import numpy as np
_30k = ["#202f66", "#ff7048", "#7f68d0", "#f3d36e", "#d869ab", "#48ADA9", "#1b262c"]

Manim Community v0.13.1

In [2]:
# manim video resolution config
config["frame_size"] = (2560, 1080)
config["frame_rate"] = 60
config["background_color"] = BLACK

In [3]:
def pad_number_text(number_arr, pad):
    number_arr_pad = []
    for i in number_arr:
        if not np.isnan(i):
            number_arr_pad.append(f"{i.astype(np.int):^{pad}}")
        else:
            number_arr_pad.append(' '*pad)
    return number_arr_pad

In [4]:
%%manim -v WARNING SieveOfEratosthenes
config["frame_width"] = 21 * 1.25
config["frame_height"] = 9 * 1.25
# config["save_last_frame"] = True

class SieveOfEratosthenes(Scene):
    
    def construct(self):
        
        self.initialize_base_table()
        
        self.base_table.scale(0.6).to_edge(DOWN, buff=1)
        self.play(self.base_table.create(run_time=1, lag_ratio=0.1))
        
        
        current_text = Text(f" ", font="Gill Sans Nova").to_edge(UP, buff=1)
        max_num_to_search = int(np.sqrt(self.n))
        
        for i in range(1, max_num_to_search):
            current_num = self.number_array[i]
            old_text = current_text
            current_text = Text(f"for i=2..{max_num_to_search}, i = {current_num}", font="Inconsolata", font_size=40).to_edge(UP, buff=1)
            current_frame = SurroundingRectangle(self.base_table.get_entries()[i], buff=0.2, color=_30k[1])
            
            # find numbers that are not divisible by current number 
            # i.e. the current number is not a factor of them
            # current number, if it is prime, it will still be active(color)
            self.flag = (self.number_array % current_num != 0) | (self.number_array == current_num)
            self.prime_flag *= self.flag
            
            # if a number is not masked as a prime number
            if self.prime_flag[i] == 1:
                algor_text = Text(f"Remove all numbers k > {current_num}, that k % {current_num} = 0", font="Inconsolata", font_size=24*1.6).next_to(current_text, DOWN)                
                self.play(ReplacementTransform(old_text, current_text, run_time=0.4), Write(current_frame, run_time=0.5), Write(algor_text, run_time=0.5))
                self.mask_not_prime(current_num)
                self.update_table_color()
            else:
                algor_text = Text(f"{current_num} is not a prime number.", font="Inconsolata", font_size=24).next_to(current_text, DOWN)
                self.play(ReplacementTransform(old_text, current_text, run_time=0.5), Write(current_frame, run_time=0.5), Write(algor_text, run_time=0.5))
            
            self.play(FadeOut(current_frame, run_time=1), FadeOut(algor_text, run_time=1))
        
        self.play(FadeOut(current_text), FadeOut(algor_text))
        self.get_all_primes()
        self.wait()

        
    def initialize_base_table(self):
        self.n = 200
        self.n_col = 20
        self.number_array = np.arange(1, self.n+1)
        self.prime_flag = np.ones_like(self.number_array)
        self.prime_flag[0] = 0
        
        # pad array
        self.n_row = int(np.ceil(self.n/self.n_col))
        padded_length = self.n_row*self.n_col
        padded_number_arr = self.number_array.copy()
        for i in range(padded_length-self.n):
            np.append(padded_number_arr, np.nan)
    
        table_text = np.array(pad_number_text(padded_number_arr, 3))
        table_text = np.reshape(table_text, (self.n_row, self.n_col))
        base_table = Table(table_text, line_config={"stroke_width": 0}, h_buff=0.5)
        base_table.get_entries().set_color(_30k[3])
        base_table.get_entries()[0].set_color("#343434")
        self.base_table = base_table
    
    def update_table_color(self):
        old_table = self.base_table
        for i in range(self.n):
            if self.prime_flag[i] == 0:
                self.base_table.get_entries()[i].set_color("#343434")
        self.play(ReplacementTransform(old_table, self.base_table, run_time=1))
    
    def mask_not_prime(self, current_num):
        cross_group = []
        for i in range(current_num, self.n):
            if self.flag[i] == 0:
                # create cross
                current_entry = self.base_table.get_entries()[i]
                cross = Cross(current_entry, stroke_color=_30k[1])
                cross_group.append(cross)
        
        self.play(FadeIn(*cross_group, run_time=1, lag_ratio=0.2))
        self.play(FadeOut(*cross_group, run_time=0.25, lag_ratio=0))
    
    def get_all_primes(self):
        prime_numbers = self.number_array * self.prime_flag
        prime_numbers = np.array(prime_numbers[prime_numbers != 0])
        n_prime = len(prime_numbers)
        
        # insert blank value, in order to be able to use reshape
        padded_length = int(np.ceil(n_prime/self.n_col))*self.n_col
        for i in range(padded_length-n_prime):
            prime_numbers = np.append(prime_numbers, np.nan)
        
        prime_padded = pad_number_text(prime_numbers, 3)
        prime_padded = np.reshape(prime_padded, (int(np.ceil(n_prime/self.n_col)), self.n_col))
        prime_table = Table(prime_padded, line_config={"stroke_width": 0}, h_buff=0.5).scale(0.8)
        prime_table.get_entries().set_color(_30k[4])
        self.play(ReplacementTransform(self.base_table, prime_table, run_time=2))
        self.wait(2)
        self.play(FadeOut(prime_table))
